In [12]:
import numpy as np
from typing import NamedTuple
from pathlib import Path
from uuid import uuid4
from shutil import copytree

from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory

In [74]:
def solution_directory(case):
    return SolutionDirectory(case.path)


def parameter_file(case, relative_path):
    return ParsedParameterFile(case.path / relative_path)


class BaseCase(NamedTuple):
    """Base case is a cleaned version of the system. If it contains any fields,
    it will only be the `0` time. Any field that is copied for manipulation will
    do so on top of an available base case in the `0` slot."""
    root: Path
    case: str
    
    @property
    def path(self):
        return self.root / self.case
    
    def new_vector(self, name=None):
        """Creates new `Vector` using this base case."""
        new_case = name or uuid4().hex
        new_path = self.root / new_case
        if not new_path.exists():
            copytree(self.path, new_path)
        return Vector(self, new_case, 0)


class Vector(NamedTuple):
    base: BaseCase
    case: str
    time: int
    
    @property
    def path(self):
        return self.base.root / self.case

In [81]:
blob_base = BaseCase(Path("./blob"), "baseCase")
a = blob_base.new_vector("a")
b = blob_base.new_vector("b")

In [82]:
x = parameter_file(a, "system/setFieldsDict")
y = parameter_file(b, "system/setFieldsDict")

In [83]:
x['regions'][1]['centre'][0] = 3
y['regions'][1]['centre'][0] = 7

In [84]:
x.writeFile()
y.writeFile()